#  CS 522 - Advanced Data Mining - Spring 2016 - Final Project

#                Gender Classification using Twitter Feeds

# Step - 5) Error Analysis.

In [1]:
import pickle
import numpy as np
from collections import Counter
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Load these tweets
tweets = pickle.load(open('tweets.pkl', 'rb'))
junk_tweets = pickle.load(open('junk_tweets.pkl', 'rb')) 

In [3]:
from sampleClass14 import sampleClass14

In [4]:
sampleClass = sampleClass14()

In [5]:
male_names, female_names = sampleClass.get_census_names()

In [7]:

tokens_list = [sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False, use_text=True,collapse_stop_words = False)
                for t in tweets]
vocabulary = sampleClass.make_vocabulary(tokens_list)
X = sampleClass.make_feature_matrix(tokens_list, vocabulary,len(tweets))

53816 unique terms in vocabulary


In [16]:
y_junk = np.array([sampleClass.get_gender(t, male_names, female_names) for t in junk_tweets])
print 'gender labels:', Counter(y_junk).items()

gender labels: [(-1, 200)]


In [12]:
y = np.array([sampleClass.get_gender(t, male_names, female_names) for t in tweets])
print 'gender labels:', Counter(y).items()

gender labels: [(0, 5182), (1, 4818)]


In [8]:
def get_terms(vocab):
    return np.array([x[0] for x in sorted(vocab.items(), key=lambda x: x[1])])

In [48]:

def save_top_terms(model, terms, n=20):
    print '\nTop Coefficients'
    save_model = model
    save_coef = model.coef_[0]
    srted = np.argsort(save_coef)

In [58]:
def get_terms_and_coef(X, tokens_list, tweets, vocabulary):
    """ Print tweets that we got wrong. """
    terms = get_terms(vocabulary)
    clf = LogisticRegression()
    train_idx = np.arange(8000)
    test_idx = np.arange(8000, len(tokens_list))
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print 'accuracy=%g' % accuracy_score(y[test_idx], predicted)
    coef = clf.coef_[0]
    srted = np.argsort(coef)
    return coef,terms
    #save_top_terms(clf, terms)

In [59]:
coef,terms = get_terms_and_coef(X, tokens_list, tweets, vocabulary)
print len(terms)
print len(coef)

accuracy=0.704
53816
53816


In [95]:
prediction = []
for t in junk_tweets:
    junk_tokens_list = sampleClass.tweet2tokens(t, use_descr=True, lowercase=True,keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False, use_text=True,collapse_stop_words = False)
    common_terms = list(set(junk_tokens_list) & set(terms))
    total_common_terms = len(list(set(junk_tokens_list) & set(terms)))
    coeff_count = 0
    for i in range(0,total_common_terms):
        coeff_count = coeff_count + coef[np.where(terms == common_terms[i])]
    if coeff_count >= 0:
        prediction.append((t['user']['name'],t['user']['description'],coeff_count,t['user']['screen_name'],'female'))
        print '\n user text: ',t['user']['name'],'\n user description: ',t['user']['description'],'\n coeff: ',coeff_count,'\n username: ',t['user']['screen_name'],'\t gender: female'
    else:
        prediction.append((t['user']['name'],t['user']['description'],coeff_count,t['user']['screen_name'],'male'))
        print '\n user text: ',t['user']['name'],'\n user description: ',t['user']['description'],'\n coeff: ',coeff_count,'\n username: ',t['user']['screen_name'],'\t gender: male'


 user text:  AJ Noel ★ 
 user description:  24. USC Alum, Basketball Junkie, Actor, Host, and Lover of all things Miami. #HeatNation 
 coeff:  [ 0.47472589] 
 username:  ImAJNoel 	 gender: female

 user text:  عبيد 
 user description:  ما أحب تويتر و شكرا.. ما بكتب مبتعث لانه راحت موضتها SnapChat: iboodi123 
 coeff:  [ 0.51020988] 
 username:  obido3 	 gender: female

 user text:  Sof 
 user description:  Plant your own gardens instead of waiting for someone to bring you flowers 
 coeff:  [ 2.50359838] 
 username:  SofiaRozo_ 	 gender: female

 user text:  BAMN 
 user description:  Stay far from timid Only make moves when ya heart's in it And live the phrase Sky's The Limit 
 coeff:  [ 1.1659177] 
 username:  Ofcourse_yess 	 gender: female

 user text:  Moe A 
 user description:  None 
 coeff:  [-0.21037032] 
 username:  moe_chi 	 gender: male

 user text:  CMPunkFan 
 user description:  CM Punk the best wrestler in the world.        3x World Champion 2x WWE Champion 
 coeff:  [ 3.129

In [99]:
f = open('predicting.txt','w')
f.write("(user_text,user_description,coefficient,username,gender)")
f.write("\n")
for i in range(0,len(prediction)):
    f.write(str(prediction[i]))
    f.write("\n---------------------------------------------------------------------------\n")

In [44]:
def print_top_terms(model, terms, n=20):
    print '\nTop Coefficients'
    coef = model.coef_[0]
    print '\ncoefff->',len(coef)
    print '\nterms->',len(terms)
    srted = np.argsort(coef)
    topi = srted[::-1][:n]
    boti = srted[:n]
    print 'Female Terms:\n' + '\n'.join('%s (%g)' % (n, c) for n, c in zip(terms[topi], coef[topi]))
    print '\nMale Terms:\n' + '\n'.join('%s (%g)' % (n, c) for n, c in zip(terms[boti], coef[boti]))
    print '\nintercept=%g' % model.intercept_

In [10]:
def print_terms_and_coef(row, terms, coef):
    indices = sorted(row.indices, key=lambda x: coef[x])
    print 'Top Terms:'
    for i in indices:
        if coef[i] != 0:
            print terms[i], coef[i]
    print

In [45]:
def error_analysis(X, tokens_list, tweets, vocabulary):
    """ Print tweets that we got wrong. """
    terms = get_terms(vocabulary)
    clf = LogisticRegression()
    train_idx = np.arange(8000)
    test_idx = np.arange(8000, len(tokens_list))
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    print 'accuracy=%g' % accuracy_score(y[test_idx], predicted)
    print_top_terms(clf, terms)
    predicted_proba = clf.predict_proba(X_test)
    print '\nERRORS:'
    for i in range(len(predicted)):
        probability = predicted_proba[i][predicted[i]]
        # If we're very wrong.
        if predicted[i] != y_test[i] and probability > .95:
            og_idx = test_idx[i]
            print '\npred=%d (%g) truth=%d \ntext=%s \ndescr=%s \nname=%s' % (predicted[i],
                                                                  probability,
                                                                  y_test[i],
                                                                  tweets[og_idx]['text'],
                                                                  tweets[og_idx]['user']['description'],
                                                                  tweets[og_idx]['user']['name'])
            print_terms_and_coef(X.getrow(og_idx), terms, clf.coef_[0])
    
error_analysis(X, tokens_list, tweets, vocabulary)

accuracy=0.704

Top Coefficients

coefff-> 53816

terms-> 53816
Female Terms:
d=girl (1.83299)
d=mom (1.68653)
d=mother, (1.67941)
d=mom, (1.53724)
d=✨ (1.49101)
d=softball (1.40983)
d=♡ (1.31117)
d=❤️ (1.27829)
d=princess (1.2105)
d=wild (1.19295)
d=under (1.17859)
d=sweet (1.13339)
d=queen (1.11177)
d=kanjiklub (1.1045)
d=someone (1.04766)
d=insta: (1.0377)
d=should (1.02199)
d=myself (1.01011)
d=#faithfulmilitarywoman (1.00657)
d=freckles. (1.00657)

Male Terms:
d=pcc (-2.13416)
d=devil (-1.90899)
d=guy (-1.88287)
d=father (-1.66223)
@thirstykirstie_ (-1.50028)
d=husband, (-1.42679)
@miranda_epley (-1.27819)
d=father, (-1.26866)
? (-1.25066)
d=husband (-1.19078)
#aldubbonvoyage (-1.13003)
d=sports (-1.10077)
d=isn't (-1.07427)
d=sometimes (-1.04967)
boy (-1.04927)
d=guy. (-1.03657)
any (-1.03496)
tx (-1.02251)
tip (-1.02116)
d=former (-0.996186)

intercept=0.176889

ERRORS:

pred=1 (0.986585) truth=0 
text=Spring fling pub crawl #chattavegas @ Big River Grille Downtown https://t.co/